##                        TEAM SOUTHERN TECHNICAL REPORT LAYOUT

1. Team Introduction.



1. Description of Solution Algorithm.
    - Features on how algorithm was implemented to get SPH Code working
    - Flowchart to highlight main simulation loops and steps required within the loops
    - How was particle leaking dealth with?



2. Demonstration of software functionality with FE time stepping method.
    - Animations included.
    - Brief description of results.



3. Demonstration of software functionality with Predictor-Corrector time stepping scheme.
    - Animations included.
    - Brief description of results.



4. Comparison between SPH simulator and shallow water velocities.
    - Discussion



5. Impact of choice of velocity of sound in water.



6. Impact of changing the smoothing length.



8. Convergence Analysis.
    - Describe error meteric used and how it was obtained.
    - What order of convergence?



9. Impact of Artifical pressure term
    - Compare results with and without artificial pressure 
    - Aminations of results obtained.



10. SPH simulator for non vertical/horizontal boundries



11. SPH simulator for a shoaling wave.



12. Challenges and Lessons learnt



13. Conclusion


# 1. Solution Algorithm


## Timestepping Algorithms

At each time step, ten particles attribute need to be updated along with the time step itself. The atrributes are:
- particle neighbours, `adj list`
- particles acceleration, or change in velocity, `a_i`
- particles change in density, `D_i`
- particles smoothing, `dW_i`
- particles relative distance, `r_ij`
- particles relative velocity to the neighbours, `v_ij`
- particles pressure, `P`

and the three attributes calculated using the Forward euler or Improved Euler scheme:
- particle's position, `x_i`
- particle's velocity, `v_i`
- particle's dencsity, `rho_i`


The following section will introduce how bothe Euler methods are implementing these changes through the system. 

### Forward Euler

The initial implementation of the SPH simulator uses a Forward Euler timestepping scheme. It timesteps the particle's position, velocity and density:

$$
{x}_{i}^{t+1} = {x}_{i}^{t} + {\Delta t}{v}_{i}^{t}\\\\
{v}_{i}^{t+1} = {v}_{i}^{t} + {\Delta t}{a}_{i}^{t}\\\\
{\rho}_{i}^{t+1} = {\rho}_{i}^{t} + {\Delta t}{D}_{i}^{t}\\\\
$$

<img src="./report_diagrams/fwd_euler.png" width=500x>
    
The Forward Euler gives promising results and allows for a good visualisation of the flow's behaviour. It however require very sdmall time step in order to work and is only first order accurate. 


### Improved Euler

The second and improved implemtation of the SPH simulator makes use of an Imporoved Euler time stepping schemes. This time stepping solution is second order accurate and takes the form: 

## Find Neighbours


## Grid Initialisation

The grid allocation was immproved to suit better with a front end code. The new gird initialisation works by running a series of methods in the main class. This new method also ensures that there are no overlapping particles.

The user input to create the grid consists of 
- x_max, upper corner of the domain
- x_min, lower corner of the domain
- f(x, y), which gives coordinates a value of 1 when a particle is present and 0 when the spaceis empty.

The f(x, y) function is written as inequalities for the case example, such as: 

```python
 def f(x, y):
    if 0 <= y <= 2 or (0 <= x <= 3 and 0 <= y <= 5):
        return 1
    else:
        return 0
```

The logic flow of the grid initialisation is then:

<img src="./report_diagrams/init_grid.png" width=500x>

where 3 method set up the grid:

```python 
def place_point()
```

which creates a particle object, gives it basic parameters and adds it to the particle list of the main class

```python 
def add_boundaries()
```

which pads the grid outside of range [x_min, x_max] and calls `place_point` to add boundaries as bound particles

```python 
def initialise grid(f(x,y))
```

which loops over all coordinates and calls `place_point` where f(x, y) is true. Once this is done it calls `add_boundaries` and checks for duplicates. 


## Particle Leakeage Control

Particle leakage in the simulation was identified as one of the main challenges of this sph simulation. Two main components were added to prevent from leakage of the system: 

### Lennard-Jones potential Boundary Forces

LJ forces were added to the calculations. This implementation adds an acceleration component to particles that are within desired distance to the wall to push them back into the domain. 

The desired distance is computed by ratio:

${q}_{ref} = \frac{{d}_{ref}}{{r}_{wall-normal}}$

where $d_{ref}$ is a set fraction of the $dx$ set by the user, and ${r}_{wall-normal}$ is the perpendicular distance between the particle and the wall.If the ratio is larger than one, the LJ force are added to the acceleration of the particle.

The acceleration component is:

$$
{a}_{wall-normal} = \frac{{P}_{ref}({q}_{ref}^{4} - ({q}_{ref}^{2})}{{r}_{wall-normal}{\rho}_{i}}
$$

and is added to the acceleration of the particle to push it back into the domain.

This boundary control was extensively tested and proved to work 


### Leakage Damage Control

This code was implemented early on the code and were used as a initial control for leakages. It consists of checking wether a particle: - has neighbours, and -is within the grid domain. If the particle does not go through this validation it is earased for the domain particle list. 


## Particle Object 

All of the particle information is save in a particle object such as any property of the particle can be called from the instance of the main sph simulator. 

The particle properties are:

``` python
self.id = next(self._ids)
self.main_data = main_data
self.x = np.array(x)
self.v = np.zeros(2)
self.a = np.zeros(2)
self.D = 0
self.rho = 0.0
self.P = 0.0
self.m = 0.0
self.bound = None
self.adj = []
```

## 

# 2. Demonstration of Functionality

## Forward Euler Method

## Improved Euler Method

## Improved Euler and Artificial Pressure Method

## Shoaling Water Method5

# 3. Improved Euler Task 

# 4. Average Wave Velocity

# 5. Artificial Pressure

- The Artificial pressure term introduced by J. J Monaghan was implemented to improve the simulation results and asthetics as follows:


- Applying the artificial pressure term to the SPH acceleration of a particle 'a' gives;


$$a_i = \sum_{j=0}^{N} m_j \left(\frac{P_i}{\rho_i^2} + \frac{P_j}{\rho_j^2}\right) \frac{dW}{dr} e_{ij} +  \mu \sum_{j=0}^{N} m_j \left(\frac{1}{\rho_i^2} + \frac{1}{\rho_j^2}\right) \frac{dW}{dr} \frac{V_i}{\left|r_{ij}\right|} + \sum_{j=0}^{N} Rf_{ij}^n + g $$


where $r > 0$,  $R = f(P,\rho)$


$$R = R_i + R_j,   \in = 0.2 $$

$$R_i = \begin{cases}\in \frac{\left|P_{i}\right|}{\rho_{i}^2} , & \text{if $P_i<0$}.\\\\
0.01\left(\frac{P_i}{\rho_i^2} + \frac{P_j}{\rho_j^2}\right) , & \text{if $otherwise$}.\end{cases}$$


To get $R_j$ we replace $i$ with $j$ in the equation above


$$W_{i,j}^n = \frac{10}{7\pi h^2}\begin{cases} \left(1 - \frac{3}{2}q^2 + \frac{3}{4}q^3\right)^n, & \text{if $0<=q<=1$}.\\\\
\frac{1}{4}\left(2 - q\right)^{3n} , & \text{if $1<=q<=2$}, \\\\
0, & \text{if $q>2$}.\end{cases}$$


$$f_{ij}^n = \frac{dW_{ij}^n}{dr}$$


$$\frac{dW_{ij}^n}{dr} = \begin{cases} \frac{40}{7\pi h^3} \left(-3q + \frac{9}{4}q\right) \left(1 - \frac{3}{2}q^2 + \frac{3}{4}q^3\right)^3, & \text{if $0<=q<=1$}.\\\\
\frac{-30}{7\pi h^3}\left(2 - q\right)^{11} , & \text{if $1<=q<=2$}, \\\\
0, & \text{if $q>2$}.\end{cases}$$

# 6. Shoaling Wave Implementation